In [1]:
import pandas as pd
import re
from pandarallel import pandarallel
from os import path
from unicodedata import normalize

In [2]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Carregar os dados

O conjunto de dados possui 1.195.278 tweets extraídos utilizando o Twitter Stream Extractor.

Aproximadamente 600k foram obtidos em 2019.

E o restante dos tweets foram obtidos durante 17 e 18 de agosto.


Dataset apenas para fins educacionais, para uso em produção recomendo a coleta durante um longo período para diminuir vieses temporais (dia dos pais, situação política, natal)

Referência:
- [Twitter Stream Extractor](https://github.com/AlanTaranti/twitter_stream_extractor)

In [3]:
# Definir caminho do arquivo
filename = "raw_data.parquet.gzip"
directory = "data"
filepath = path.join(directory, filename)

# Carregar dados
data = pd.read_parquet(filepath)

# Amostra dos dados
data.sample(10)

,id,text
550420,1559989009291649024,@aluizioikk Você finalmente vai usar a fantasi...
557530,1559990843678887938,@euleoguedes Adoro
679762,1160631667276603392,te encontrei até qm fim 🎶
725530,1160686364104564736,Pelo demoro 😴
563410,1559992343859699712,@heyitspotter_ @_miguel_mf BRIGADE FR😭
313645,1559901819740082179,Tive um sonho bom hoje 💖
369987,1559915868494221313,@hggandelman @corelinajones @misfitsvampira @V...
555714,1559990387225370624,Se viene Ouahabinha masterclass https://t.co/N...
201953,1559869498651451394,@uarmyego Parabens!!!! 😐
913567,1160916327407771648,"Brigado, meu eu leonino agradece... 😛"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195278 entries, 0 to 1195277
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1195278 non-null  object
 1   text    1195278 non-null  object
dtypes: object(2)
memory usage: 18.2+ MB


# Limpeza

Aqui o objetivo é tratarmos os dados para remover tweets duplicados e remover do tweet informações que não serão úteis para análise de sentimentos.

## Remover duplicatas

In [5]:
duplicados = data["id"].duplicated().sum()

print("Quantidade de Tweets duplicados: {}".format(duplicados))

Quantidade de Tweets duplicados: 383


In [6]:
data = data[data["id"].duplicated(keep="first") == False]

## Tratamento do Texto

In [7]:
def clean_data(string):

    # Normalizar a string
    string_tratada = normalize("NFC", string)

    # Converter para minuscula
    string_tratada = string_tratada.lower()

    # Remover referencia a outros usuários
    string_tratada = re.sub("(@[\w]+ | @[\w]+)", " ", string_tratada)

    # Remover new line
    string_tratada = re.sub("\n", " ", string_tratada)

    # Remover as hashtags
    string_tratada = re.sub("(#.+ | #.+)", " ", string_tratada)

    # Remover os links
    string_tratada = re.sub("\w+:\/\/\S+", " ", string_tratada)

    # Remover Numeros
    string_tratada = re.sub("\d", " ", string_tratada)

    # Remover underline
    string_tratada = re.sub("_", " ", string_tratada)

    # Tratar Repetições — Permite no máximo 3
    string_tratada = re.sub(r"(.+?)\1{3,}", r"\1\1\1", string_tratada)

    # Remover espaços extras
    string_tratada = re.sub(" +", " ", string_tratada)
    string_tratada = re.sub("(^ )", "", string_tratada)
    string_tratada = re.sub("( $)", "", string_tratada)

    return string_tratada

## Demonstração de função de limpeza de dados

In [8]:
tweet = data["text"].sample(1).values[0]
tweet_limpo = clean_data(tweet)

print("Tweet Original : {}".format(tweet))
print("")
print("Tweet Limpo    : {}".format(tweet_limpo))

Tweet Original : @jauremodest uhum 😔

Tweet Limpo    : uhum 😔


## Aplicação da Limpeza

In [9]:
data["text"] = data["text"].parallel_apply(clean_data)

# Salvar Dados

In [10]:
# Definir caminho do arquivo
filename = "cleaned_data.parquet.gzip"
directory = "data"
filepath = path.join(directory, filename)

# Salvar Dados
data.to_parquet(filepath, compression="gzip")